# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

#### Import the necessary libraries

In [1]:
from nltk.tag import pos_tag
import nltk
from sklearn_crfsuite import CRF, metrics
from sklearn.metrics import make_scorer,confusion_matrix
from pprint import pprint
from sklearn.metrics import f1_score,classification_report
from sklearn.pipeline import Pipeline
import spacy
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
nlp = spacy.load('en_core_web_sm')
import string
import re
import pickle

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [2]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [3]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [4]:
def tuples_maker(input_sigs, output_labels):
    result = []
    for input_sig, output_label in zip(input_sigs, output_labels):
        sentence = []
        for token, label in zip(input_sig, output_label):
            if label.startswith("Duration"):
                sentence.append((token, label))
            elif label.startswith("Period"):
                sentence.append(("every", "EVERY"))
                sentence.append((token, label))
            else:
                sentence.append((token.lower(), label))
        result.append(sentence)
    return result

In [5]:
result_tup = tuples_maker(input_sigs, output_labels)
print(result_tup)

[[('for', 'FOR'), ('5', 'Duration'), ('to', 'TO'), ('6', 'DurationMax'), ('days', 'DurationUnit')], [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')], [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')], [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')], [('every', 'EVERY'), ('every', 'EVERY'), ('day', 'Period')], [('every', 'EVERY'), ('every', 'EVERY'), ('2', 'Period'), ('every', 'EVERY'), ('weeks', 'PeriodUnit')], [('every', 'EVERY'), ('every', 'EVERY'), ('3', 'Period'), ('every', 'EVERY'), ('days', 'PeriodUnit')], [('every', 'EVERY'), ('every', 'EVERY'), ('1', 'Period'), ('to', 'TO'), ('every', 'EVERY'), ('2', 'PeriodMax'), ('every', 'EVERY'), ('months', 'PeriodUnit')], [('every', 'EVERY'), ('every', 'EVERY'), ('2', 'Period'), ('to', 'TO'), ('every', 'EVERY'), ('6', 'PeriodMax'), ('every', 'EVERY'), ('weeks', 'PeriodUnit')], [('every', 'EVERY'), ('every', 'EVERY'), ('4', 'Period'), ('to', 'TO'), ('every', 'EVERY'), ('6', 'PeriodMax'), ('every', 'EVERY'), ('d

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [6]:


def triples_maker(result_tup):
    
    result = []
    for tuple_list in result_tup:
        sentence = []
        for token, label in tuple_list:
            doc = nlp(token) 
            pos_tag = doc[0].tag_ 
            sentence.append((token, pos_tag, label))
        result.append(sentence)
    return result




In [7]:
sample_data = triples_maker(result_tup)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'VB', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'VBD', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'VBD', 'FOR'),
  ('3', 'LS', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('every', 'DT', 'EVERY'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('every', 'DT', 'EVERY'),
  ('3', 'LS', 'Period'),
  ('every', 'DT', 'EVERY'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('every', 'DT', 'EVERY'),
  ('2', 'CD', 'PeriodMax'),
  ('every', 'DT', 'EVERY'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EV

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [8]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'word.has_digit=%s' % any(char.isdigit() for char in word),
        'word.has_punct=%s' % any(char in string.punctuation for char in word),
        'word.is_uppercase=%s' % word.isupper(),
        'word.is_lowercase=%s' % word.islower(),
        'word.is_titlecase=%s' % word.istitle()
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features


### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [9]:
train_features = []
train_labels = []
for sentence in sample_data:
    sentence_features = []
    sentence_labels = []
    for i in range(len(sentence)):
        token_features = token_to_features(sentence, i)
        sentence_features.append(token_features)
        sentence_labels.append(sentence[i][2])
    train_features.append(sentence_features)
    train_labels.append(sentence_labels)


#need to make a list of lists in bith train_features and train_labels
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

train_features


[[['bias',
   'word.lower=for',
   'word[-3:]=for',
   'word[-2:]=or',
   'word.isupper=False',
   'word.istitle=False',
   'word.isdigit=False',
   'postag=IN',
   'word.has_digit=False',
   'word.has_punct=False',
   'word.is_uppercase=False',
   'word.is_lowercase=True',
   'word.is_titlecase=False',
   'BOS',
   '+1:word.lower=5',
   '+1:word.istitle=False',
   '+1:word.isupper=False',
   '+1:word.isdigit=True',
   '+1:postag=CD'],
  ['bias',
   'word.lower=5',
   'word[-3:]=5',
   'word[-2:]=5',
   'word.isupper=False',
   'word.istitle=False',
   'word.isdigit=True',
   'postag=CD',
   'word.has_digit=True',
   'word.has_punct=False',
   'word.is_uppercase=False',
   'word.is_lowercase=False',
   'word.is_titlecase=False',
   '-1:word.lower=for',
   '-1:word.istitle=False',
   '-1:word.isupper=False',
   '-1:word.isdigit=False',
   '-1:postag=IN',
   '+1:word.lower=to',
   '+1:word.istitle=False',
   '+1:word.isupper=False',
   '+1:word.isdigit=False',
   '+1:postag=TO'],
  ['bia

In [10]:
len(train_features)

56

In [11]:
train_labels

[['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'],
 ['Method', 'Qty', 'Form'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['EVERY', 'EVERY', 'Period'],
 ['EVERY', 'EVERY', 'Period', 'EVERY', 'PeriodUnit'],
 ['EVERY', 'EVERY', 'Period', 'EVERY', 'PeriodUnit'],
 ['EVERY',
  'EVERY',
  'Period',
  'TO',
  'EVERY',
  'PeriodMax',
  'EVERY',
  'PeriodUnit'],
 ['EVERY',
  'EVERY',
  'Period',
  'TO',
  'EVERY',
  'PeriodMax',
  'EVERY',
  'PeriodUnit'],
 ['EVERY',
  'EVERY',
  'Period',
  'TO',
  'EVERY',
  'PeriodMax',
  'EVERY',
  'PeriodUnit'],
 ['Method', 'Qty', 'TO', 'Qty', 'Form'],
 ['Method', 'Qty', 'TO', 'Qty', 'Form'],
 ['Method',
  'Qty',
  'Form',
  'PO',
  'BID',
  'FOR',
  'Duration',
  'DurationUnit',
  'AT',
  'WHEN'],
 ['Method', 'Qty', 'Form', 'TID', 'PO'],
 ['Method',
  'Qty',
  'Form',
  'PO',
  'EVERY',
  'EVERY',
  'Period',
  'EVERY',
  'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'],
 ['Method

### Training the CRF model with the features extracted using the feature extractor method

In [12]:
import sklearn_crfsuite
trainer = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    
)


# Submit training data to the trainer
trainer.fit(X_train, y_train)

# Set the parameters of the model

with open('presc_parser.pkl', 'wb') as f:
    pickle.dump(trainer, f)
# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished


### Predicting the test data with the built model

In [13]:
y_pred=trainer.predict(X_test)

In [14]:
y_pred_test=trainer.predict(["take 2 tabs every 6 hours x 10 days"])

In [15]:
y_pred_test

[['Method',
  'Qty',
  'Form',
  'Frequency',
  'FOR',
  'Duration',
  'DurationUnit',
  'Method',
  'Qty',
  'Form',
  'Frequency',
  'FOR',
  'Duration',
  'DurationUnit',
  'Method',
  'Qty',
  'Form',
  'Frequency',
  'FOR',
  'Duration',
  'DurationUnit',
  'Method',
  'Qty',
  'Form',
  'Frequency',
  'FOR',
  'Duration',
  'DurationUnit',
  'Method',
  'Qty',
  'Form',
  'Frequency',
  'FOR',
  'Duration',
  'DurationUnit']]

In [16]:
y_test

[['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'],
 ['EVERY', 'EVERY', 'Period', 'EVERY', 'PeriodUnit'],
 ['BID'],
 ['Method', 'Qty', 'Form', 'TID', 'PO'],
 ['EVERY',
  'EVERY',
  'Period',
  'TO',
  'EVERY',
  'PeriodMax',
  'EVERY',
  'PeriodUnit'],
 ['EVERY', 'EVERY', 'Period', 'EVERY', 'PeriodUnit'],
 ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'],
 ['Qty', 'Form', 'Frequency', 'EVERY', 'Period', 'EVERY', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'],
 ['Method',
  'Qty',
  'Form',
  'PO',
  'BID',
  'FOR',
  'Duration',
  'DurationUnit',
  'AT',
  'WHEN'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['FOR', 'Duration', 'DurationUnit']]

In [17]:
f1 = metrics.flat_f1_score(y_test, y_pred, average='weighted')

In [18]:
f1

0.8672839506172839

### Putting all the prediction logic inside a predict method

In [19]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
 
    tokens = nltk.word_tokenize(sig)
    sentence_features = [token_to_features([(token, '')], 0) for token in tokens]
    predictions=trainer.predict([sentence_features])[0]
    print(sig)
    print(predictions)
    return predictions

### Sample predictions

In [20]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
['Method', 'Qty', 'Form', 'EVERY', 'PeriodMax', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


In [21]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [22]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [23]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']


In [24]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']


In [25]:
predictions = predict("x 30 days")

x 30 days
['FOR', 'Duration', 'DurationUnit']


In [26]:
predictions = predict("x 20 months")

x 20 months
['Method', 'Qty', 'Form']


In [27]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']


In [28]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
['Method', 'Qty', 'Form', 'PO', 'EVERY', 'PeriodMax', 'PeriodUnit']


In [29]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
['Method', 'Qty', 'Form', 'Frequency', 'TID']


In [30]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']


In [31]:
predictions = predict("inject 3 units orally")

inject 3 units orally
['Method', 'Qty', 'Form', 'Frequency']


In [32]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
['Method', 'Method', 'Qty', 'Form', 'TID']


In [33]:
predictions = predict("by mouth take three caps")

by mouth take three caps
['BY', 'PO', 'Method', 'Qty', 'Form']


In [34]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
['Method', 'Qty', 'Form', 'Method', 'Qty', 'TIMES', 'EVERY', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [35]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
['Method', 'Qty', 'Form', 'Method', 'Qty', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [36]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
['Method', 'Qty', 'Form', 'EVERY', 'PeriodUnit', 'AT', 'WHEN']


In [37]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']


In [38]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
['Method', 'Qty', 'Form', 'Frequency', 'QID', 'Method', 'Qty', 'Form']


In [39]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form', 'AND', 'Method', 'Qty', 'Form']


In [40]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
['Method', 'Method', 'Qty', 'Form']


In [41]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']


In [42]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


In [43]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'Method', 'FOR', 'Duration', 'DurationUnit']


In [44]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
['Method', 'Qty', 'Form']


In [45]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
['Method', 'Qty', 'TO', 'Qty', 'Form']


In [46]:
prediction = predict("take two to four tabs")

take two to four tabs
['Method', 'Qty', 'TO', 'Qty', 'Form']


In [47]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'Duration', 'DurationUnit']


In [48]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [49]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [50]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [51]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [52]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [53]:
prediction = predict("every 60 mins")

every 60 mins
['EVERY', 'Period', 'M']


In [54]:
prediction = predict("every 10 mins")

every 10 mins
['EVERY', 'Duration', 'M']


In [55]:
prediction = predict("every two to four months")

every two to four months
['EVERY', 'PeriodUnit', 'TO', 'Qty', 'Form']


In [56]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [57]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
['EVERY', 'Period', 'TO', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form']


In [58]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
['Method', 'Qty', 'EVERY', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form']


In [59]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
['Method', 'Qty', 'Form', 'Frequency', 'Method', 'EVERY', 'Duration', 'DurationUnit']


In [60]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
['Method', 'Method', 'Qty', 'Form', 'EVERY', 'Duration', 'DurationUnit']


In [61]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [62]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [102]:
prediction = predict("10 tabs")

10 tabs
['Qty', 'Form']
